In [1]:
# Initial imports
import re
import pandas as pd
import numpy as np

difusion_col = ['Caso ID ', 'Fecha Programada ', 'Difusión ']
caso_col = ['Difusión ', 'Caso ID ', 'Código de Error ', 'Mensaje de Error ', 'Parámetros Extra ']
message_col = ['Fecha ','Tipo Mensaje', 'Asunto ','Grupo ', 'Contenido ', 'Caso ID ', 'Desde Cuenta ', 'A Cuenta ']

path_base = r'C:/Users/Usuario/Documents/b4d/data/raw/'
file_path = 'tigoev_s1_difusiones'
file_path_1 = 'tigoev_s1_message'
file_ext = '.csv'

df_difusion = pd.read_csv(path_base + file_path + file_ext, usecols = difusion_col)
df_caso = pd.read_csv(path_base + file_path + file_ext, usecols = caso_col)
df_message = pd.read_csv(path_base + file_path_1 + file_ext, usecols = message_col, delimiter=";")

df_difusion.rename(columns={'Fecha Programada ': 'scheduled_date', 'Difusión ': 'difusion_name', 'Caso ID ': 'caso_id',}, inplace=True)
df_caso.rename(columns={'Difusión ': 'difusion_name', 'Caso ID ': 'caso_id', 'Código de Error ': 'error_code', 'Mensaje de Error ': 'error_msg', 'Parámetros Extra ': 'params_extra'}, inplace=True)
df_message.rename(columns={'Fecha ': 'msg_datetime', 'Tipo Mensaje': 'msg_tp', 'Asunto ': 'msg_subject ', 'Contenido ': 'msg_content', 'Caso ID ': 'caso_id', 'Grupo ': 'msg_group', 'Desde Cuenta ': 'from_acc', 'A Cuenta ': 'to_acc'}, inplace=True)

In [2]:
# Classification difusion
def clasificate_difusion(data_str):
    patterns_map = {
        r"Primer envio de Producci'on|Nicolas Martinez - B4D|RE: Prueba Melvin|Primer envió|Prueba_interna_v2|prueba_base_II|Prueba Melvin": "Difusion 1 15072024",
        r"Base II - Tigo EV|Base II - Tigo EV Envio 2|Base II - Tigo EV - Envio 3": "Difusion 2 120824 1200",
        r"BASE III - Envio 1 AM|BASE III - Envio 2 AM|BASE III - Envio 4 AM": "Difusion 3 AM 220824 1000",
        r"BASE III - Envio 3 - pm": "Difusion 4 PM 230824 1900",
        r"Prueba Interna III|Enivo III - Producción|Envio III - Producción AM|Envio III - Envio AM 2|Prueba IV|": "Difusion 5 Persona Natural Fijo 160924 10000",
        r"RE: preuba melvin tigo el salvador|Envio 6 \| Natural Fijo|preuba melvin tigo el salvador|RE: RE: preuba melvin tigo el salvador|RE: RE: RE: preuba melvin tigo el salvador|difusiones prueba cambio plan natural movil|RE: difusion_tigoev_natural_movil_prueba_281124|difusion_tigoev_natural_movil_prueba_281124|difusion pruebas tigo ev redireccion":"Difusion 6 Cambio Plan Natural Movil 051224 1630",
        r"DifusionTest150125|DifusionTestHFC": "Difusion 7 Clientes HFC 150125 1000",
        r"difusion_1230_lote1|difusion_movil_1230_lote2|difusion_movil_1230_lote3|difusion_movil_1230_lote4|difusion_movil_1230_lote5": "Difusion 8 Natural Movil 1230 210125",
        r"difusion_grupo_1700_lote1|difusion_1700_movil_lote2|difusion_1700_movil_lote3|difusion_1700_movil_lote4|difusion_lote_1700_lote5": "Difusion 8 Natural Movil 1700 210125"
    }
    for pattern, result in patterns_map.items():
        if re.fullmatch(pattern, data_str):
            return result
    return "unknown"


df_difusion['difusion_name_base'] = df_difusion['difusion_name'].apply(lambda x: clasificate_difusion(x))

df_difusion['difusion_name_upper'] = df_difusion['difusion_name'].str.upper()

regex_1 = r'NICOLAS MARTINEZ - B4D|PREUBA|PRUEBA|PRUEBAS|PRIMER ENVIÓ'

df_difusion['tipo'] = np.where(df_difusion['difusion_name_upper'].str.contains(regex_1, case=True, regex=True, na=False), 'PRUEBA', 'LOTE')

#df_difusion.groupby(['sdifusion_name_base','scheduled_date'])
df_difusion_tmp = df_difusion[df_difusion["tipo"]!="PRUEBA"]

df_difusion_tmp['scheduled_date'] = df_difusion_tmp.groupby(['difusion_name_base'])['scheduled_date'].transform('first')

df_difusion_tmp.drop(columns=['difusion_name', 'difusion_name_upper'], inplace=True)

df_difusion_tmp.drop_duplicates(inplace=True)

df_difusion = df_difusion.merge(df_difusion_tmp, on='caso_id', how='left')
df_difusion.drop(columns=['difusion_name_base_y', 'difusion_name_upper', 'tipo_y'], inplace=True)

df_difusion.rename(columns={'scheduled_date_x': 'scheduled_date', 'scheduled_date_y': 'scheduled_date_main', 'difusion_name_base_x': 'difusion_name_base', 'tipo_x': 'tipo'}, inplace=True)

df_difusion['scheduled_date_main'] = df_difusion.groupby(['difusion_name_base'])['scheduled_date_main'].transform('first')
df_difusion['scheduled_date_main'] = pd.to_datetime(df_difusion['scheduled_date_main'])
df_difusion['date'] = df_difusion['scheduled_date_main'].dt.date
df_difusion['hour'] = df_difusion['scheduled_date_main'].dt.hour
df_difusion['minute'] = df_difusion['scheduled_date_main'].dt.minute

df_difusion = df_difusion[['caso_id', 'difusion_name', 'difusion_name_base', 'tipo', 'scheduled_date', 'scheduled_date_main', 'date', 'hour', 'minute']]
# Merge difusiones y casos
df_difusion_caso = df_caso.merge(df_difusion, on='caso_id', how='left')
df_difusion_caso = df_difusion_caso[['caso_id', 'difusion_name_y', 'difusion_name_base', 'tipo', 'scheduled_date', 'scheduled_date_main', 'date', 'hour', 'minute', 'error_code', 'error_msg']]
df_difusion_caso.rename(columns={'difusion_name_y': 'difusion_name'}, inplace=True)
# Only difusion message
df_message = df_message.merge(df_difusion_caso, how='left', on='caso_id')[['caso_id',  'from_acc', 'to_acc', 'msg_datetime', 'msg_tp', 'msg_subject ', 'msg_content', 'msg_group', 'difusion_name', 'difusion_name_base', 'tipo', 'scheduled_date', 'scheduled_date_main', 'date', 'hour', 'minute', 'error_code', 'error_msg']]

# change datatype
df_message['msg_datetime'] = pd.to_datetime(df_message['msg_datetime'])

df_message['msg_date'] = df_message['msg_datetime'].dt.date

# change datatype 
df_message['error_msg'] = df_message['error_msg'].astype(str)

# change datatype
df_message['msg_content'] = df_message['msg_content'].astype(str)

df_message['cstmr_nmbr'] = np.where(df_message['msg_tp'] == 'Entrante', df_message['from_acc'], df_message['to_acc'])

df_message.sort_values(by=['cstmr_nmbr', 'difusion_name_base', 'msg_datetime'], inplace=True)

# filtramos solo difusiones
# Ya no filtramos por que si un usuario responde despues del tiempo configurado, se cuenta como un nuevo caso
#df_msg = df_msg[df_msg['msg_group']=='Bot Anfitrion']

# filtramos solo salientes
#df_msg = df_msg[df_msg['msg_tp']=='Saliente']

# Quitando la columna contenidos null
#df_msg = df_message[df_message['msg_content'].notna()]

df_message = df_message[['caso_id', 'cstmr_nmbr', 'msg_datetime', 'msg_tp', 'msg_subject ', 'msg_content', 'msg_group', 'difusion_name', 'difusion_name_base', 'tipo', 'scheduled_date', 'scheduled_date_main', 'date', 'hour', 'minute', 'error_code', 'error_msg']]

df_message['difusion_name_base'] = df_message['difusion_name_base'].ffill()
df_message['difusion_name'] = df_message['difusion_name'].ffill()
df_message['tipo'] = df_message['tipo'].ffill()
df_message['scheduled_date'] = df_message['scheduled_date'].ffill()
df_message['scheduled_date_main'] = df_message['scheduled_date_main'].ffill()
df_message['date'] = df_message['date'].ffill()
df_message['hour'] = df_message['hour'].ffill()
df_message['minute'] = df_message['minute'].ffill()
df_message['error_code'] = df_message['error_code'].ffill()
df_message['error_msg'] = df_message['error_msg'].ffill()

# Quitando la columna contenidos null
df_message = df_message[df_message['difusion_name'].notna()]


df_message['error_msg'] = np.where(df_message['error_msg']=="nan", "null", df_message['error_msg']) 

C:\Users\Usuario\AppData\Local\Temp\ipykernel_25176\1370523398.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_difusion_tmp['scheduled_date'] = df_difusion_tmp.groupby(['difusion_name_base'])['scheduled_date'].transform('first')
C:\Users\Usuario\AppData\Local\Temp\ipykernel_25176\1370523398.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_difusion_tmp.drop(columns=['difusion_name', 'difusion_name_upper'], inplace=True)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_25176\1370523398.py:35: SettingWithCopyWarning: 
A value is trying to be set on a cop

In [ ]:

df_message_tmp = df_message.copy()
# map errores
map_error = {
    r'Undeliverable|undeliverable': 'No entregable',
    r'experiment'                 : 'Numero Experimental'
}

def f_mapeo_error(response):
    if response is None:
        return -1
    
    for pattern, nodo in map_error.items():
        if re.search(pattern, response, re.IGNORECASE):
            return nodo
    return '-1'

df_message_tmp['error_tp'] = df_message_tmp['error_msg'].apply(lambda x: f_mapeo_error(x))
# Mapeo nodos
nodos_res = {
    r'^\¡En Tigo Business impulsamos|^\¡hola|En Tigo Business'   : '1', # Esta interesado
    r'^Aumenta los beneficios|\¡Excelente|^Elige el plan que sea|\:one\: \*Opci|\:one\:\*Opci'      : '2',
    r'^Yo|^recibe|confirmo que deseo recibir|^\:gift: recibe'                              : '3', # Primera condicion
    r'Favor ingresar su numero de DUI \(Sin Guiones\) para validar su identidad|^\¡Gracias por elegir el plan|^\:gift\: recibe'         : '4', # Segunda condicion  
    r'^\*\¡GRACIAS POR CONFIAR EN NOSOTROS|^\¡Gracias|^\!Todo listo,|en las próximas las próximas 24 horas':'5',
    r'^\¿Estas seguro'       : '51',
    r'^Recuerda que'        : '61',
    r'^Aún estas'           : '62',
    r'^Por favor, continua con el proceso': '71',
    r'^\*\¡Estamos para|Estamos para ayudarte'       : '81',    # Ayuda
    r'^DUI ingresado incorrecto|Favor ingresar número de DUI válido|El número de DUI ingresado': '91', # DUI inconrrecto
    r'Actualmente no tienes una oferta': '91', # DUI inconrrecto
    r'Servicio al Cliente Tigo Busines': '6'
}

def f_mapeo_nodo(response, map_data):
    for pattern, nodo in map_data.items():
        if re.search(pattern, response, re.IGNORECASE):
            return nodo
    return '-1'

df_message_tmp['nodo'] = df_message_tmp[df_message_tmp['msg_tp'] == 'Saliente']['msg_content'].apply(lambda x: f_mapeo_nodo(x, nodos_res))
df_msg_nodo = df_message_tmp.groupby('nodo', as_index=False).size()

df_message_tmp_1 = df_message_tmp.copy()

df_message_tmp_1['msg_date_init'] = df_message_tmp_1.groupby(['cstmr_nmbr', 'difusion_name_base'], as_index=False)['msg_datetime'].transform('first') 
df_message_tmp_1['msg_date_last'] = df_message_tmp_1.groupby(['cstmr_nmbr', 'difusion_name_base'], as_index=False)['msg_datetime'].transform('last') 

df_message_tmp_1['caso_id'] = df_message_tmp_1['caso_id'].astype(str)
df_message_tmp_2 = df_message_tmp_1.groupby(['cstmr_nmbr', 'difusion_name_base', 'tipo','msg_date_init', 'msg_date_last','error_tp'], as_index=False).agg({'nodo':lambda x: list(x), 'caso_id': lambda x: '_'.join(set(x))})
df_msg_user = df_message_tmp[df_message_tmp['msg_tp']=='Entrante']

# Mapeo respuestas
nodos_response = {
    r'Si, estoy interesado'   : '1',
    r'No estoy Interesado|No estoy interesado'    : '2',
    r'monto|iva|costo|barato|precio' : '3',
    r'nombre|mal nombre|papa' : '4',
    r'necesito|solo necesito': '5',
    r'no necesito': '6',
    r'\d+': '7',
    r'Opción 1' : '21',
    r'Opción 2' : '22',
    r'Opción 3' : '23',
    r'(Gracias por|Bienvenido|En breve|En un momento|¿Cómo podemos|podemos ayudarte|no podemos responder|te atenderemos|Nuestros horarios|horario de atención|Le saluda|:emoji)': '90'
}

def f_mapeo_nodo_1(response, map_data):
    for pattern, nodo in map_data.items():
        if re.search(pattern, response, re.IGNORECASE):
            return nodo
    return '-1'

df_msg_user['answer'] = df_msg_user['msg_content'].apply(lambda x: f_mapeo_nodo_1(x, nodos_response))
df_msg_user['msg_date_init'] = df_msg_user.groupby(['cstmr_nmbr', 'difusion_name_base'], as_index=False)['msg_datetime'].transform('first') 
df_msg_user['msg_date_last'] = df_msg_user.groupby(['cstmr_nmbr', 'difusion_name_base'], as_index=False)['msg_datetime'].transform('last') 
# 
df_msg_user['caso_id'] = df_msg_user['caso_id'].astype(str)

df_msg_tmp_1 = df_msg_user.groupby(['cstmr_nmbr', 'difusion_name_base', 'tipo', 'msg_date_init', 'msg_date_last', 'error_tp'], as_index=False).agg({'answer':lambda x: list(x), 'caso_id': lambda x: '_'.join(set(x))})
df_msg_tmp_1.rename(columns={'answer': 'answer_flow'}, inplace=True)
df_msg_tmp_1.columns = ['cstmr_nmbr', 'difusion_name_base', 'tipo','user_msg_date_init', 'user_msg_date_last', 'user_error_tp', 'answer_flow', 'caso_id']

df_msg_fct = df_message_tmp_2.merge(df_msg_tmp_1[['cstmr_nmbr', 'difusion_name_base', 'tipo','user_msg_date_init', 'user_msg_date_last', 'answer_flow', 'caso_id']], on=['cstmr_nmbr', 'difusion_name_base'], how='left') 
df_msg_fct = df_msg_fct[['cstmr_nmbr', 'difusion_name_base', 'tipo_x', 'msg_date_init', 'msg_date_last', 'error_tp', 'nodo', 'caso_id_x', 'user_msg_date_init', 'user_msg_date_last', 'answer_flow']]

#  'caso_id'



C:\Users\Usuario\AppData\Local\Temp\ipykernel_25176\1112718457.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_msg_user['answer'] = df_msg_user['msg_content'].apply(lambda x: f_mapeo_nodo_1(x, nodos_response))
C:\Users\Usuario\AppData\Local\Temp\ipykernel_25176\1112718457.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_msg_user['msg_date_init'] = df_msg_user.groupby(['cstmr_nmbr', 'difusion_name_base'], as_index=False)['msg_datetime'].transform('first')
C:\Users\Usuario\AppData\Local\Temp\ip

In [69]:

def f_scene(data):
    if '5' in data:
        return 'Conversion'
    elif '5' not in data and len(data) > 1:
        return 'No termino el proceso'
    return 'No inicio el proceso'


df_msg_fct['scene'] = df_msg_fct['nodo'].map(lambda x: f_scene(x))

def f_last_nodo(data):
    if '5' in data and len(data) > 1:
        return '5 - Confirmacion de la activacion'
    elif '4' in data and len(data) > 1:
        return '4 - Verificacion DUI'
    elif '3' in data and len(data) > 1:
        return '3 - Aceptacion de la oferta'
    elif '2' in data and len(data) > 1:
        return '2 - Seleccion de las ofertas'
    elif '1' in data:
        return '1 - Datos actuales del cliente'
    return ''

df_msg_fct['last_nodo'] = df_msg_fct['nodo'].map(lambda x: f_last_nodo(x))

df_msg_fct['interest'] = df_msg_fct['answer_flow'].apply(lambda x: '1' in x if isinstance(x, list) else False) 

df_msg_fct['nodo_1_to_2'] = df_msg_fct['nodo'].apply(lambda x: '1' and '2' in x if isinstance(x, list) else False) 

df_msg_fct['nodo_2_to_3'] = df_msg_fct['nodo'].apply(lambda x: '2' and '3' in x if isinstance(x, list) else False) 

df_msg_fct['nodo_3_to_4'] = df_msg_fct['nodo'].apply(lambda x: '3' and '4' in x if isinstance(x, list) else False) 

df_msg_fct['nodo_4_to_5'] = df_msg_fct['nodo'].apply(lambda x: '4' and '5' in x if isinstance(x, list) else False) 

df_msg_fct['nodo_5_to_6'] = df_msg_fct['nodo'].apply(lambda x: '5' and '6' in x if isinstance(x, list) else False) 

df_msg_fct['nodo_1'] = df_msg_fct['nodo'].apply(lambda x: '1' in x if isinstance(x, list) else False) 

df_msg_fct['nodo_2'] = df_msg_fct['nodo'].apply(lambda x: '2' in x if isinstance(x, list) else False) 

df_msg_fct['nodo_3'] = df_msg_fct['nodo'].apply(lambda x: '3' in x if isinstance(x, list) else False) 

df_msg_fct['nodo_4'] = df_msg_fct['nodo'].apply(lambda x: '4' in x if isinstance(x, list) else False) 

df_msg_fct['nodo_5'] = df_msg_fct['nodo'].apply(lambda x: '5' in x if isinstance(x, list) else False) 




In [70]:
df_msg_fct['msg_date_answer_days'] = np.where(df_msg_fct['user_msg_date_init'] >= df_msg_fct['msg_date_init'], (df_msg_fct['user_msg_date_init'] - df_msg_fct['msg_date_init']) / pd.Timedelta(days=1), None)

df_msg_fct['tiempo_respuesta_segundos'] = (df_msg_fct['user_msg_date_init'] - df_msg_fct['msg_date_init']).dt.total_seconds()

def clasificar_tiempo(diferencia_dias):
    if diferencia_dias is None:
        return 'Sin respuesta'
    
    if diferencia_dias < 0:
        return 'Respuesta antes del envío'
    elif diferencia_dias == 0:
        return 'Mismo día'
    elif diferencia_dias < 1:
        return '< 1 día'
    elif diferencia_dias < 2:
        return '< 2 días'
    elif diferencia_dias < 3:
        return '< 3 días'
    else:
        return '> 3 días o más'
    
df_msg_fct['time_response'] = df_msg_fct['msg_date_answer_days'].apply(clasificar_tiempo)

df_msg_fct['ayuda'] = df_msg_fct['nodo'].apply(lambda x: '81' in x if isinstance(x, list) else False)

df_msg_fct['bot'] = df_msg_fct['answer_flow'].apply(lambda x: '90' in x if isinstance(x, list) else False) 

df_msg_fct['timeout_response'] = df_msg_fct['answer_flow'].apply(lambda x: '61' or '62' in x if isinstance(x, list) else False) 
df_msg_fct['no_response'] = df_msg_fct['answer_flow'].apply(lambda x: '71' in x if isinstance(x, list) else False) 
df_msg_fct['conversion'] = (df_msg_fct['tipo_x'] == 'LOTE') & (df_msg_fct['scene'] == 'Conversion')

In [71]:
# reorder
#df_message = df_message[['caso_id', 'cstmr_nmbr', 'difusion_name', 'difusion_name_base', 'msg_datetime', 'msg_date', 'msg_tp', 'msg_subject', 'msg_content', 'msg_group', 'error_msg', 'cstmr_nmbr', 'error_tp', 'nodo', 'answer']]
df_message['msg_date'] = pd.to_datetime(df_message['date'])
df_message['scheduled_date'] = pd.to_datetime(df_message['scheduled_date_main'])
df_message['msg_datetime'] = pd.to_datetime(df_message['msg_datetime'])

df_message.to_excel('../data/tigo_esv_message_4.xlsx') 

In [72]:

df_msg_fct['msg_date_init'] = pd.to_datetime(df_msg_fct['msg_date_init'])
df_msg_fct['msg_date_last'] = pd.to_datetime(df_msg_fct['msg_date_last'])
df_msg_fct['user_msg_date_init'] = pd.to_datetime(df_msg_fct['user_msg_date_init'])
df_msg_fct['user_msg_date_last'] = pd.to_datetime(df_msg_fct['user_msg_date_last'])

df_msg_fct.to_excel('../data/tigo_esv_caso_msg_fct_4.xlsx')


In [51]:
df_msg_nodo = df_message.groupby('nodo', as_index=False).size()

KeyError: 'nodo'